# Indeed web scraping project

In [1]:
import csv
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
from selenium import webdriver

# Starting up the webdriver

In [28]:
driver = webdriver.Chrome()

In [29]:
url = 'https://in.indeed.com'
driver.get(url)

In [30]:
def get_url(position, location):
    template = 'https://in.indeed.com/jobs?q={}&l={}&from=searchOnHP&vjk=96800a585102f7fa'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    return template.format(position, location)

In [31]:
driver.get(get_url('data analyst', 'noida'))

# Extracting the data

In [7]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [8]:
results = soup.find_all('div', {'class' : 'slider_item css-kyg8or eu4oa1w0'})

In [9]:
len(results)

15

# Prototype the record

In [10]:
item = results[0]

atag = item.h2.a
job_title = atag.text
print(job_title)



Data Analyst


In [11]:
company_name = item.find('span', 'companyName').text

In [12]:
try:
    rating_of_company = item.find('span', 'ratingNumber').text
except AttributeError:
    rating_of_company = ''

In [13]:
date_of_posting = item.find('span', 'date').text

In [14]:
today = datetime.today().strftime('%Y-%m-%d')

In [15]:
try:
    salary_range = item.find('div', 'attribute_snippet').text
except AttributeError:
    salary_range = ''

# Generalize the pattern

In [16]:
def get_record(item):
    
    atag = item.h2.a
    job_title = atag.text
    
    company_name = item.find('span', 'companyName').text
    
    try:
        rating_of_company = item.find('span', 'ratingNumber').text
    except AttributeError:
        rating_of_company = ''
        
        
    date_of_posting = item.find('span', 'date').text   
    today = datetime.today().strftime('%Y-%m-%d')
    
    try:
        salary_range = item.find('div', 'attribute_snippet').text
    except AttributeError:
        salary_range = ''
        
  
    
    record = (job_title, company_name, rating_of_company, date_of_posting, today, salary_range)
    
    return record
        
        

In [17]:
records = []

results = soup.find_all('div', {'class' : 'slider_item css-kyg8or eu4oa1w0'})

for item in results:
    records.append(get_record(item))


In [18]:
records[0]

('Data Analyst',
 'Zones',
 '2.8',
 'PostedPosted 5 days ago',
 '2023-01-03',
 'Full-time')

# Getting to the next page

In [33]:
while True:
    try:
        url = 'https://in.indeed.com' + soup.find('a', {'aria-label' : 'Next Page'}).get('href')
    except AttributeError:
        break
        
        
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    results = soup.find_all('div', {'class' : 'slider_item css-kyg8or eu4oa1w0'})
    
    for item in results:
        records.append(get_record(item))

In [23]:
len(records)

330

# Putting it all together

In [26]:
import csv
from bs4 import BeautifulSoup
from datetime import datetime

from selenium import webdriver

def get_url(position, location):
    template = 'https://in.indeed.com/jobs?q={}&l={}&from=searchOnHP&vjk=96800a585102f7fa'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    return template.format(position, location)


def get_record(item):
    
    atag = item.h2.a
    job_title = atag.text
    
    company_name = item.find('span', 'companyName').text
    
    try:
        rating_of_company = item.find('span', 'ratingNumber').text
    except AttributeError:
        rating_of_company = ''
        
        
    date_of_posting = item.find('span', 'date').text   
    today = datetime.today().strftime('%Y-%m-%d')
    
    try:
        salary_range = item.find('div', 'attribute_snippet').text
    except AttributeError:
        salary_range = ''
        
  
    
    record = (job_title, company_name, rating_of_company, date_of_posting, today, salary_range)
    
    return record



def main(position, location):
    
    records = []
    url = get_url(position, location)
    
    while True:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'class' : 'slider_item css-kyg8or eu4oa1w0'})
        
        for item in results:
            records.append(get_record(item))
      
    
        try:
            url = 'https://in.indeed.com' + soup.find('a', {'aria-label' : 'Next Page'}).get('href')
        except AttributeError:
            break
        
        
    driver.close()   
    
           
    # save the job data
    
    with open('indeedposting.csv', 'w', newline='', encoding='UTF-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Job_title', 'Company_name', 'Rating_of_company', 'Date_of_posting', 'Today', 'Salary_range'])
        writer.writerows(records)
        
    


In [32]:
main('data analyst', 'noida')